In [1]:
%load_ext autoreload
%autoreload 2

# TODO: there really must be a better way to do this
import sys
import os
sys.path.append("../")

# change this line to the where directory in which the GitHub repository is located: datasets will be installed into 
# the directory one level above this
os.chdir("/lustre_scratch/orlando-code/coralshift/")

## Getting HAR transformer running on my data

In [4]:
from coralshift.machine_learning.transformer.transformer import TSTransformerEncoderClassiregressor
from coralshift.machine_learning.trainer.trainer import Trainer
from coralshift.machine_learning.transformer_utils import get_data

from torch.optim import Adam

In [ ]:
# get data. N.B. only works when connected to Jupyter server
# dh = get_data(train_path='lustre_scratch/datasets/har_test/MotionSenseHAR_TRAIN.ts',
#                 test_path='lustre_scratch/datasets/har_test/MotionSenseHAR_TEST.ts')

# dh = get_data()

In [ ]:
# TODO: parameter sweep on lr, feat_dim, d_model, max_len, n_heads, num_layers, dim_feedforward, num_classes, dropout, 
# pos_encoding, activation, norm, freeze 
for lr in [1e-4]:
    model = TSTransformerEncoderClassiregressor(
        feat_dim=12,
        d_model=64,
        max_len=1000,
        n_heads=8,
        num_layers=6,
        dim_feedforward=512,
        num_classes=6,
        dropout=0.1,
        pos_encoding="learnable",
        activation="gelu",
        norm="BatchNorm",
        freeze=False,
    )
    trainer = Trainer(dh=dh, epochs=1)
    dh.create_dataset()
    dh.split_data(train_split=0.8)
    dataloader_train = dh.create_dataloader(dh.train_data, batch_size=8)
    optimiser = torch.optim.Adam(model.parameters(), lr=lr)
    trainer.fit(dataloader=dataloader_train, model=model, optimiser=optimiser)
    dataloader_test = dh.create_dataloader(dh.test_data, batch_size=8)
    accuracy = trainer.evaluate(dataloader=dataloader_test, model=model)
    print(accuracy)
